In [1]:
%%capture
import sys
!{sys.executable} -m pip install pycryptodome

In [9]:
%%capture
import random
import math
import string
import numpy as np

In [3]:
CERT_AUTH = { 'e': 65537, 'N': 33790420484761320225234266446986435791020053290995177788399417698648848366075439013295931744537889745793682732187585867814285806211190774412138926826806937374931229955338241741978503726324443629746710612128866806815968501932728765477787763877641403710570749219182260822344263730489611164845428107854720086677}

In [5]:
class Server:
    RSA_PUBLIC_KEY = (
        65537,
        9038494040587010144527283006157928881319808198037800262507236135386268739708006022679916006469762927636026303078260115925689450875819797050237830498287577
                     )
    
    RSA_PRIVATE_KEY = 2535417770757764232372393774283341693305817384267313426399332119458339022396333163725804069143378584578179197504985944189947045947523212247400435285708353
    
    SIGNED_SERVER_CERTIFICATE = 12700340844416036644598467941678481034795616611285372846483555754936821137468034013552250333992213649482476234812345812443409067513447778032927158222423442969388095714455462826384483204764801308373583214095146933570191105218603640338334545613351980554082339092228528550094782514990171966181657825617151691397
    
    ELLIPTIC_CURVE = [0,7]
    
    ELLIPTIC_CURVE_MODULUS = 0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffc2f
    
    GENERATOR_POINT = (0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
       0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8)
    
    GENERATOR_POINT_ORDER = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141
    
    LOGINS = {}
    
    def __init__(self):
        pass
    
    def signMessage(self, message):
        return signRSA(self.RSA_PRIVATE_KEY, message, self.RSA_PUBLIC_KEY[1])
    
    def publishRSAPublicKey(self):
        """
        Returns ((e, N), signature) where e,N are the server's public key, and signature is the cert authorities' signature 
        on e||N.
        """
        
        return ((self.RSA_PUBLIC_KEY[0], self.RSA_PUBLIC_KEY[1]), self.SIGNED_SERVER_CERTIFICATE)
    
    def generateECDHParameters(self):
        """
        Returns a generator point P, curve E, and modulus N signed via the server's RSA keys.
        """

        intToSign = int(str(self.ELLIPTIC_CURVE[0]) + str(self.ELLIPTIC_CURVE[1]) 
                        + str(self.ELLIPTIC_CURVE_MODULUS) + str(self.GENERATOR_POINT[0]) + str(self.GENERATOR_POINT[1]))

        signature = signRSA(self.RSA_PRIVATE_KEY, intToSign, self.RSA_PUBLIC_KEY[1])

        return self.ELLIPTIC_CURVE, self.ELLIPTIC_CURVE_MODULUS, self.GENERATOR_POINT, self.GENERATOR_POINT_ORDER, signature
    
    def generateSignedECDHMessage(self):
        nP, n = generateECDH(self.GENERATOR_POINT, self.ELLIPTIC_CURVE, self.ELLIPTIC_CURVE_MODULUS, self.GENERATOR_POINT_ORDER)
        
        intToSign = int(str(nP[0]) + str(nP[1]))
        
        self.ECDH_SECRET = n
        
        return nP, self.signMessage(intToSign)
    
    def verifyECDHMessage(self, P, signature, publicKey):
        """
        Verifies point P was signed by the given publicKey (e, N).
        """

        expected = int(str(P[0]) + str(P[1]))

        return verifyRSA(publicKey[0], signature, publicKey[1], expected)
    
    def acceptConnection(self, clientPublicKey):
        self.CLIENT_PUBLIC_KEY = clientPublicKey
    
    def acceptECDHMessage(self, P, signature):
        """
        Given the sender's public point, verify its signature is as expected and then update internal state
        """
        
        assert self.verifyECDHMessage(P, signature, self.CLIENT_PUBLIC_KEY), "Invalid signature on received ECDH message."
        
        # Signature verified
        sharedSecretPoint = doubleAndAdd(P, self.ECDH_SECRET, self.ELLIPTIC_CURVE, self.ELLIPTIC_CURVE_MODULUS)
        
        self.SHARED_SECRET_INT = pointToMessage(sharedSecretPoint)
        
        return True
    
    def createLogin(self, username, password):
        if username in self.LOGINS: 
            return False
        
        salt = ''.join(np.random.choice(list(string.ascii_lowercase), 5))
        
        intToHash = textToInt(password + salt)
        
        hashedPassword = strangeHashFunction(intToHash)
        
        self.LOGINS[username] = (hashedPassword, salt)
        
        return True
        
    def verifyLogin(self, username, password):
        assert username in self.LOGINS, "Username does not exist"
        
        hashedPassword = self.LOGINS[username][0]
        salt = self.LOGINS[username][1]
        
        intToHash = textToInt(password + salt)
                
        return hashedPassword == strangeHashFunction(intToHash)
    
    def encryptMessage(self, message):
        """
        Given string message, encrypts with the current setup
        """
        
        return encryptAES(self.SHARED_SECRET_INT, bytes(message))
    
    def decryptMessage(self, encrypted):
        """
        Given encrypted message, decrypts with the current setup
        """
        
        return decryptAES(self.SHARED_SECRET_INT, encrypted)
    
    def verifyRequestIntegrity(self, requestedData):
        """
        Takes in a request of the form { data: ..., hmac: ...} and returns True if the HMAC is valid.
        """
        
        return verifyHMAC(self.SHARED_SECRET_INT, bytes(requestedData['data'], 'utf-8'), requestedData['hmac'])
    
    def handleRequest(self, requestData):
        if 'data' not in requestData or 'hmac' not in requestData:
            raise AssertionError("Invalid requested format")
        assert self.verifyRequestIntegrity(requestData), "Request MAC check failed"
        
        # Request verified, time to decrypt
        request = json.loads(self.decryptMessage(requestData['data']))
        
        if request['type'] == 'createLogin':
            assert 'username' in request and 'password' in request, "Bad request data"
            
            return self.createLogin(request['username'], request['password'])
        elif request['type'] == 'verifyLogin':
            return self.verifyLogin(request['username'], request['password'])
        else:
            raise AssertionError("Invalid request type")